In [36]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error

In [30]:
df_sample_submission = pd.read_csv('./lentahack/20210521_sample_submission.csv')
df_sample_submission

,Offer_ID,UpLift
0,386,0
1,182,0
2,173,0
3,375,0
4,379,0
...,...,...
144,536,0
145,99,0
146,100,0
147,95,0


In [2]:
data = pd.read_csv('data.csv')
data

,Offer_ID,number_of_products,diversity_1,diversity_2,diversity_3,diversity_4,duration,promotype,uplift,most_freq_1,most_freq_2,most_freq_3,most_freq_4,train_test_group,pct_change
0,10,8,0.0,0.0,0.000000,0.000000,13,1.0,15.941591,1.0,21.0,191.0,515.0,train,0.531065
1,100,12,0.0,0.0,0.000000,1.418296,13,3.0,NaN,0.0,56.0,45.0,1103.0,test,-0.040186
2,101,1,0.0,0.0,0.000000,0.000000,13,3.0,1.761594,1.0,27.0,107.0,951.0,train,0.460020
3,102,11,0.0,0.0,0.000000,0.000000,13,3.0,0.630470,1.0,26.0,31.0,1790.0,train,1.115112
4,103,1,0.0,0.0,0.000000,0.000000,13,3.0,2.505152,1.0,33.0,136.0,1766.0,train,0.243666
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
884,F95,1,0.0,0.0,0.000000,0.000000,6,2.0,2.169574,1.0,28.0,14.0,868.0,train,0.227658
885,F96,22,0.0,0.0,0.000000,3.174674,6,2.0,0.401636,1.0,49.0,333.0,1888.0,train,0.372452
886,F97,14,0.0,0.0,0.000000,0.985228,6,2.0,2.276210,1.0,17.0,362.0,948.0,train,1.158110
887,F98,3,0.0,0.0,0.918296,0.918296,6,2.0,1.593010,1.0,23.0,334.0,769.0,train,0.083164


In [3]:
data['train_test_group'].value_counts()

train    740
test     149
Name: train_test_group, dtype: int64

In [4]:
df_train = data[data['train_test_group'] == 'train']
df_test = data[data['train_test_group'] == 'test']

df_train

,Offer_ID,number_of_products,diversity_1,diversity_2,diversity_3,diversity_4,duration,promotype,uplift,most_freq_1,most_freq_2,most_freq_3,most_freq_4,train_test_group,pct_change
0,10,8,0.0,0.0,0.000000,0.000000,13,1.0,15.941591,1.0,21.0,191.0,515.0,train,0.531065
2,101,1,0.0,0.0,0.000000,0.000000,13,3.0,1.761594,1.0,27.0,107.0,951.0,train,0.460020
3,102,11,0.0,0.0,0.000000,0.000000,13,3.0,0.630470,1.0,26.0,31.0,1790.0,train,1.115112
4,103,1,0.0,0.0,0.000000,0.000000,13,3.0,2.505152,1.0,33.0,136.0,1766.0,train,0.243666
5,104,2,0.0,0.0,0.000000,0.000000,13,3.0,-0.254974,0.0,62.0,39.0,430.0,train,0.923554
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
884,F95,1,0.0,0.0,0.000000,0.000000,6,2.0,2.169574,1.0,28.0,14.0,868.0,train,0.227658
885,F96,22,0.0,0.0,0.000000,3.174674,6,2.0,0.401636,1.0,49.0,333.0,1888.0,train,0.372452
886,F97,14,0.0,0.0,0.000000,0.985228,6,2.0,2.276210,1.0,17.0,362.0,948.0,train,1.158110
887,F98,3,0.0,0.0,0.918296,0.918296,6,2.0,1.593010,1.0,23.0,334.0,769.0,train,0.083164


In [12]:
X_train = df_train[['number_of_products','diversity_1','diversity_2','diversity_3','diversity_4',\
                  'duration','promotype','most_freq_1','most_freq_2','most_freq_3','most_freq_4','pct_change']].values
y_train = df_train['uplift'].values
print(X_train.shape)
print(y_train.shape)

(740, 12)
(740,)


In [16]:
X_holdout = df_test[['Offer_ID','number_of_products','diversity_1','diversity_2','diversity_3','diversity_4',\
                  'duration','promotype','most_freq_1','most_freq_2','most_freq_3','most_freq_4','pct_change']].values
X_holdout.shape

(149, 13)

In [38]:
# gridsearchcv on svr
param = {'kernel':('linear','rbf','sigmoid'),'C':[1,5,10],'coef0':[0.01,10,0.5],'gamma':('auto','scale')}
modelsvr = SVR()
grids = GridSearchCV(modelsvr,param,cv=3,verbose=3)
grids.fit(X_train,y_train)

Fitting 3 folds for each of 54 candidates, totalling 162 fits
[CV 1/3] END .....C=1, coef0=0.01, gamma=auto, kernel=linear; total time=  39.1s
[CV 2/3] END .....C=1, coef0=0.01, gamma=auto, kernel=linear; total time=  35.9s
[CV 3/3] END .....C=1, coef0=0.01, gamma=auto, kernel=linear; total time=  34.8s
[CV 1/3] END ........C=1, coef0=0.01, gamma=auto, kernel=rbf; total time=   0.0s
[CV 2/3] END ........C=1, coef0=0.01, gamma=auto, kernel=rbf; total time=   0.0s
[CV 3/3] END ........C=1, coef0=0.01, gamma=auto, kernel=rbf; total time=   0.0s
[CV 1/3] END ....C=1, coef0=0.01, gamma=auto, kernel=sigmoid; total time=   0.0s
[CV 2/3] END ....C=1, coef0=0.01, gamma=auto, kernel=sigmoid; total time=   0.0s
[CV 3/3] END ....C=1, coef0=0.01, gamma=auto, kernel=sigmoid; total time=   0.0s
[CV 1/3] END ....C=1, coef0=0.01, gamma=scale, kernel=linear; total time=  38.5s
[CV 2/3] END ....C=1, coef0=0.01, gamma=scale, kernel=linear; total time=  35.9s
[CV 3/3] END ....C=1, coef0=0.01, gamma=scale, 

GridSearchCV(cv=3, estimator=SVR(),
             param_grid={'C': [1, 5, 10], 'coef0': [0.01, 10, 0.5],
                         'gamma': ('auto', 'scale'),
                         'kernel': ('linear', 'rbf', 'sigmoid')},
             verbose=3)

In [39]:
pred = grids.predict(x_te)
mean_absolute_error(y_te, pred)

1.2039339182922837

In [18]:
# naive
x_tr, y_tr = X_train[:592], y_train[:592]
x_te, y_te = X_train[592:], y_train[592:]
clf = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=.1)
clf.fit(x_tr,y_tr)
pred = clf.predict(x_te)
mean_absolute_error(y_te, pred)

SVR(C=100, gamma=0.1)

In [41]:
final_pred = grids.predict(X_holdout[:,1:])
df_pred = pd.DataFrame({'Offer_ID':X_holdout[:,0], 'UpLift':final_pred})
df_final = pd.merge(df_sample_submission, df_pred, on=['Offer_ID'], how='left')
df_final.drop(columns=['UpLift_x'],inplace=True)
df_final.columns = ['Offer_ID','UpLift']
df_final.to_csv('submission_7_svr.csv',index=False)